# Fooooooooooooooocus
Colab NoteBook Created by [licyk](https://github.com/licyk)

Jupyter Notebook 仓库：[licyk/sd-webui-all-in-one](https://github.com/licyk/sd-webui-all-in-one)

这是适用于 [Colab](https://colab.research.google.com) 部署 Fooocus 的 Jupyter Notebook，使用时请按顺序执行 Jupyter Notebook 单元。

Colab 链接：<a href="https://colab.research.google.com/github/licyk/sd-webui-all-in-one/blob/main/fooocus_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 功能
1. 环境配置：配置安装的 PyTorch 版本、内网穿透的方式，并安装 [Fooocus](https://github.com/lllyasviel/Fooocus)，默认设置下已启用`配置环境完成后立即启动 Fooocus`选项，则安装完成后将直接启动 Fooocus。
2. 下载模型：下载可选列表中的模型（可选）。
3. 自定义模型下载：使用链接下载模型（可选）。
4. 启动 Fooocus：启动 Fooocus，并显示访问地址。


## 使用
1. 在 Colab -> 代码执行程序 > 更改运行时类型 -> 硬件加速器 选择`GPU T4`或者其他 GPU。
2. `环境配置`单元中的选项通常不需要修改，保持默认即可。
3. 运行`环境配置`单元，默认设置下已启用`配置环境完成后立即启动 Fooocus`选项，则环境配置完成后立即启动 Fooocus，此时将弹出 Google Drive 授权页面，根据提示进行操作。配置完成后将启动 Fooocus，Fooocus 的访问地址可在日志中查看。
4. 如果未启用`配置环境完成后立即启动 Fooocus`选项，需要运行`启动 Fooocus`单元，此时将弹出 Google Drive 授权页面，根据提示进行操作。配置完成后将启动 Fooocus，Fooocus 的访问地址可在日志中查看。
5. 提示词查询工具：[SD 绘画提示词查询](https://licyk.github.io/t/tag)

## 提示
1. Colab 在关机后将会清除所有数据，所以每次重新启动时必须运行`环境配置`单元才能运行`启动 Fooocus`单元。
2. [Ngrok](https://ngrok.com) 内网穿透在使用前需要填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取。
3. 每次启动 Colab 后必须运行`环境配置`单元，才能运行`启动 Fooocus`单元启动 Fooocus。
4. 其他功能有自定义模型下载等功能，根据自己的需求进行使用。
5. 运行`启动 Fooocus`时将弹出 Google Drive 访问授权提示，根据提示进行授权。授权后，使用 Fooocus 生成的图片将保存在 Google Drive 的`fooocus_output`文件夹中。
6. Gradio 的内网穿透地址可在启动 Fooocus 后日志中的`Running on public URL`查看。
7. Fooocus 启动预设文件可自行编写，并上传到 Github 或者其他平台并获取下载链接后，可以替换`Fooocus 启动预设文件地址`中的链接，进行环境配置时将使用该预设文件启动 Fooocus。预设文件可自行下载下来进行参考，其他有关说明可阅读：[Customization - lllyasviel/Fooocus](https://github.com/lllyasviel/Fooocus?tab=readme-ov-file#customization)。
8. 在 Colab 的文件浏览器中，可临时上传 LoRA 模型，在`Fooocus`->`models`中找到`loras`文件夹，右键可以打开上传文件菜单，上传 LoRA 模型完成后在 Fooocus 中刷新模型列表即可使用，其他模型同理。注意，Colab 在关机后将会清除数据。

In [ ]:
#@title 👇 环境配置
# Fooocus Manager 功能初始化部分, 通常不需要修改
# 如果需要查看完整代码实现, 可阅读: https://github.com/licyk/sd-webui-all-in-one/blob/main/sd_scripts_ipynb_core.py
#################################################################################################################
# SD_SCRIPTS_IPYNB_CORE_URL, FORCE_DOWNLOAD_CORE 参数可根据需求修改, 通常保持默认即可
SD_SCRIPTS_IPYNB_CORE_URL = "https://github.com/licyk/sd-webui-all-in-one/raw/main/sd_scripts_ipynb_core.py" # Fooocus Manager 核心下载地址
FORCE_DOWNLOAD_CORE = False # 设置为 True 时, 即使 Fooocus Manager 已存在也会重新下载
#################################################################################################################
import os
import sys
import urllib.request
from pathlib import Path
try:
    print(f"[Manager] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}") # type: ignore
except Exception as _:
    JUPYTER_ROOT_PATH = Path(os.getcwd())
    sys.path.append(str(JUPYTER_ROOT_PATH.resolve()))
    print(f"[Manager] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}")
    SD_SCRIPTS_IPYNB_CORE_PATH = JUPYTER_ROOT_PATH / "sd_scripts_ipynb_core.py" # Fooocus Manager 核心保存路径
try:
    if SD_SCRIPTS_IPYNB_CORE_PATH.exists() and not FORCE_DOWNLOAD_CORE:
        print("[Manager] Fooocus Manager 核心模块已存在")
    else:
        urllib.request.urlretrieve(SD_SCRIPTS_IPYNB_CORE_URL, SD_SCRIPTS_IPYNB_CORE_PATH)
        print("[Manager] Fooocus Manager 核心模块下载成功")
except Exception as e:
    raise Exception(f"Fooocus Manager 核心模块下载错误: {e}")
from sd_scripts_ipynb_core import logger, VERSION, FooocusManager
logger.info("Fooocus Manager 版本: %s", VERSION)
logger.info("核心模块初始化完成")

#######################################################

#@markdown ## Fooocus 核心配置选项
#@markdown - Fooocus 分支仓库地址：
FOOOCUS_REPO = "https://github.com/lllyasviel/Fooocus" #@param {type:"string"}
#@markdown - Fooocus 启动预设文件地址：
FOOOCUS_PRESET = "https://github.com/licyk/term-sd/releases/download/archive/fooocus_config.json" #@param {type:"string"}
#@markdown - Fooocus 路径配置地址：
FOOOCUS_PATH_CONFIG = "https://github.com/licyk/term-sd/releases/download/archive/fooocus_path_config_colab.json" #@param {type:"string"}
#@markdown - Fooocus 翻译文件下载地址：
FOOOCUS_TRANSLATION = "https://github.com/licyk/term-sd/releases/download/archive/fooocus_zh_cn.json" #@param {type:"string"}
#@markdown - Fooocus 依赖表名：
FOOOCUS_REQUIREMENT = "requirements_versions.txt" #@param {type:"string"}
#@markdown - Fooocus 启动参数：
FOOOCUS_LAUNCH_ARGS = "--preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram" #@param {type:"string"}
#@markdown ---
#@markdown ## PyTorch 组件版本选项：
#@markdown - PyTorch：
PYTORCH_VER = "torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124" #@param {type:"string"}
#@markdown - xFormers：
XFORMERS_VER = "xformers==0.0.29.post3" #@param {type:"string"}
#@markdown ---
#@markdown ## 包管理器选项：
#@markdown - 使用 uv 作为 Python 包管理器
USE_UV = True #@param {type:"boolean"}
#@markdown - PyPI 主镜像源
PIP_INDEX_MIRROR = "https://pypi.python.org/simple" #@param {type:"string"}
#@markdown - PyPI 扩展镜像源
PIP_EXTRA_INDEX_MIRROR = "https://download.pytorch.org/whl/cu124" #@param {type:"string"}
#@markdown - PyTorch 镜像源
PYTORCH_MIRROR = "https://download.pytorch.org/whl/cu124" #@param {type:"string"}
#@markdown ---
#@markdown ## 内网穿透选项：
#@markdown - 使用 remote.moe 内网穿透
USE_REMOTE_MOE = True #@param {type:"boolean"}
#@markdown - 使用 localhost.run 内网穿透
USE_LOCALHOST_RUN = True #@param {type:"boolean"}
#@markdown - 使用 pinggy.io 内网穿透
USE_PINGGY_IO = True #@param {type:"boolean"}
#@markdown - 使用 CloudFlare 内网穿透
USE_CLOUDFLARE = True #@param {type:"boolean"}
#@markdown - 使用 Gradio 内网穿透
USE_GRADIO = True #@param {type:"boolean"}
#@markdown - 使用 Ngrok 内网穿透（需填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取）
USE_NGROK = True #@param {type:"boolean"}
#@markdown - Ngrok Token
NGROK_TOKEN = ""  #@param {type:"string"}
#@markdown ---
#@markdown ## 快速启动选项：
#@markdown - 配置环境完成后立即启动 Fooocus（并挂载 Google Drive）
QUICK_LAUNCH = True #@param {type:"boolean"}
#@markdown - 不重复配置环境（当重复运行环境配置时将不会再进行安装）
NO_REPEAT_CONFIGURE_ENV = True #@param {type:"boolean"}
#@markdown ---
#@markdown ## 其他选项：
#@markdown - 预下载模型时使用的下载器
MODEL_DOWNLOADER = "mix" #@param ["aria2", "requests", "mix"]
#@markdown - 预下载模型时下载线程
DOWNLOAD_MODEL_THREAD = 16 #@param {type:"slider", min:1, max:64, step:1}
#@markdown - 清理无用日志
CLEAR_LOG = True #@param {type:"boolean"}
#@markdown - 检查可用 GPU
CHECK_AVALIABLE_GPU = True #@param {type:"boolean"}
#@markdown - 启用 TCMalloc 内存优化
ENABLE_TCMALLOC = True #@param {type:"boolean"}
#@markdown - 启用 CUDA Malloc 显存优化
ENABLE_CUDA_MALLOC = True #@param {type:"boolean"}

#######################################################

INSTALL_PARAMS = {
    "torch_ver": PYTORCH_VER or None,
    "xformers_ver": XFORMERS_VER or None,
    "use_uv": USE_UV,
    "pypi_index_mirror": PIP_INDEX_MIRROR or None,
    "pypi_extra_index_mirror": PIP_EXTRA_INDEX_MIRROR or None,
    # Colab 的环境暂不需要以下镜像源
    # "pypi_find_links_mirror": PIP_FIND_LINKS_MIRROR or None,
    # "github_mirror": GITHUB_MIRROR or None,
    # "huggingface_mirror": HUGGINGFACE_MIRROR or None,
    "pytorch_mirror": PYTORCH_MIRROR or None,
    "fooocus_repo": FOOOCUS_REPO or None,
    "fooocus_requirment": FOOOCUS_REQUIREMENT or None,
    "fooocus_preset": FOOOCUS_PRESET or None,
    "fooocus_path_config": FOOOCUS_PATH_CONFIG or None,
    "fooocus_translation": FOOOCUS_TRANSLATION or None,
    "model_downloader": MODEL_DOWNLOADER,
    "download_model_thread": DOWNLOAD_MODEL_THREAD,
    "check_avaliable_gpu": CHECK_AVALIABLE_GPU,
    "enable_tcmalloc": ENABLE_TCMALLOC,
    "enable_cuda_malloc": ENABLE_CUDA_MALLOC,
}

TUNNEL_PARAMS = {
    "use_ngrok": USE_NGROK,
    "ngrok_token": NGROK_TOKEN or None,
    "use_cloudflare": USE_CLOUDFLARE,
    "use_remote_moe": USE_REMOTE_MOE,
    "use_localhost_run": USE_LOCALHOST_RUN,
    "use_gradio": USE_GRADIO,
    "use_pinggy_io": USE_PINGGY_IO,
    "message": "##### Fooocus 访问地址 #####",
}

FOOOCUS_PATH = "/content/Fooocus"
fooocus = FooocusManager(os.path.dirname(FOOOCUS_PATH), os.path.basename(FOOOCUS_PATH), port=7865)

try:
    _ = fooocus_has_init # type: ignore
except:
    fooocus_has_init = False

if NO_REPEAT_CONFIGURE_ENV:
    if not fooocus_has_init:
        fooocus.install(**INSTALL_PARAMS)
        INIT_CONFIG = 1
        fooocus_has_init = True
        CLEAR_LOG and fooocus.utils.clear_up()
        logger.info("Fooocus 运行环境配置完成")
    else:
        logger.info("检测到不重复配置环境已启用, 并且在当前 Colab 实例中已配置 Fooocus 运行环境, 不再重复配置 Fooocus 运行环境")
        logger.info("如需在当前 Colab 实例中重新配置 Fooocus 运行环境, 请在快速启动选项中取消不重复配置环境功能")
else:
    fooocus.install(**INSTALL_PARAMS)
    INIT_CONFIG = 1
    CLEAR_LOG and fooocus.utils.clear_up()
    logger.info("Fooocus 运行环境配置完成")

if QUICK_LAUNCH:
    logger.info("启动 Fooocus 中")
    os.chdir(FOOOCUS_PATH)
    fooocus.mount_drive()
    fooocus.tun.start_tunnel(**TUNNEL_PARAMS)
    logger.info("Fooocus 启动参数: %s", FOOOCUS_LAUNCH_ARGS)
    if USE_GRADIO:
        !python "{FOOOCUS_PATH}/launch.py" {FOOOCUS_LAUNCH_ARGS} --share
    else:
        !python "{FOOOCUS_PATH}/launch.py" {FOOOCUS_LAUNCH_ARGS}
    CLEAR_LOG and fooocus.utils.clear_up()
    logger.info("已关闭 Fooocus")

In [ ]:
#@title 👇 下载模型（可选）

try:
    _ = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

# 模型下载
logger.info("下载模型中")
#@markdown 选择下载的模型：
##############################


sd_xl_base = False  #@param {type:"boolean"}
sd_xl_base and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_base_1.0_0.9vae.safetensors")

anima_pencil_xl = False  #@param {type:"boolean"}
anima_pencil_xl and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animaPencilXL_v200.safetensors")

bluePencilXL  = False  #@param {type:"boolean"}
bluePencilXL and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/bluePencilXL_v401.safetensors")

AnythingXL_xl = False  #@param {type:"boolean"}
AnythingXL_xl and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/AnythingXL_xl.safetensors")

abyssorangeXLElse_v10 = False  #@param {type:"boolean"}
abyssorangeXLElse_v10 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/abyssorangeXLElse_v10.safetensors")

CounterfeitXL  = False  #@param {type:"boolean"}
CounterfeitXL and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/CounterfeitXL-V1.0.safetensors")

animeIllustDiffusion  = False  #@param {type:"boolean"}
animeIllustDiffusion and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animeIllustDiffusion_v061.safetensors")

nekorayxl = False  #@param {type:"boolean"}
nekorayxl and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/nekorayxl_v06W3.safetensors")

animagine_XL_3 = False  #@param {type:"boolean"}
animagine_XL_3 and fooocus.get_sd_model("https://huggingface.co/cagliostrolab/animagine-xl-3.0/resolve/main/animagine-xl-3.0.safetensors")

animagine_XL_3_1 = False  #@param {type:"boolean"}
animagine_XL_3_1 and fooocus.get_sd_model("https://huggingface.co/cagliostrolab/animagine-xl-3.1/resolve/main/animagine-xl-3.1.safetensors")

animagine_XL_4 = True  #@param {type:"boolean"}
animagine_XL_4 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-4.0.safetensors")

heartOfAppleXL_v20 = False  #@param {type:"boolean"}
heartOfAppleXL_v20 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v20.safetensors")

heartOfAppleXL_v30 = False  #@param {type:"boolean"}
heartOfAppleXL_v30 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v30.safetensors")

holodayo_xl_21 = False  #@param {type:"boolean"}
holodayo_xl_21 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/holodayo-xl-2.1.safetensors")

kivotos_xl_20 = False  #@param {type:"boolean"}
kivotos_xl_20 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kivotos-xl-2.0.safetensors")

sd_xl_anime_V52 = False  #@param {type:"boolean"}
sd_xl_anime_V52 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_anime_V52.safetensors")

kohakuXLGamma_rev1  = False  #@param {type:"boolean"}
kohakuXLGamma_rev1 and fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-gamma/resolve/main/kohaku-xl-gamma-rev1.safetensors")

kohakuXLBeta_beta7  = False  #@param {type:"boolean"}
kohakuXLBeta_beta7 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohakuXLBeta_beta7.safetensors")

Kohaku_XL_Delta = False  #@param {type:"boolean"}
Kohaku_XL_Delta and fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Delta/resolve/main/kohaku-xl-delta-rev1.safetensors")

kohakuXL_Epsilon = False  #@param {type:"boolean"}
kohakuXL_Epsilon and fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon/resolve/main/kohaku-xl-epsilon-rev1.safetensors")

kohakuXL_Epsilon_rev2 = False  #@param {type:"boolean"}
kohakuXL_Epsilon_rev2 and fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon-rev2/resolve/main/kohaku-xl-epsilon-rev2.safetensors")

kohakuXL_Epsilon_rev3 = False  #@param {type:"boolean"}
kohakuXL_Epsilon_rev3 and fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon-rev3/resolve/main/kohaku-xl-epsilon-rev3.safetensors")

kohakuXL_Zeta = False  #@param {type:"boolean"}
kohakuXL_Zeta and fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Zeta/resolve/main/kohaku-xl-zeta.safetensors")

starryXLV52 = False  #@param {type:"boolean"}
starryXLV52 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/starryXLV52_v52.safetensors")

BArtstyleDB = False  #@param {type:"boolean"}
BArtstyleDB and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/baxlBartstylexlBlueArchiveFlatCelluloid_xlv1.safetensors")

BArtstyleDB_3 = False  #@param {type:"boolean"}
BArtstyleDB_3 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/baxlBlueArchiveFlatCelluloidStyle_xlv3.safetensors")

pony_v6 = False  #@param {type:"boolean"}
pony_v6 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/ponyDiffusionV6XL_v6StartWithThisOne.safetensors")

pdForAnime_v20 = False  #@param {type:"boolean"}
pdForAnime_v20 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/pdForAnime_v20.safetensors")

tPonynai3_v51 = False  #@param {type:"boolean"}
tPonynai3_v51 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/tPonynai3_v51WeightOptimized.safetensors")

omegaPonyXLAnime_v20 = False  #@param {type:"boolean"}
omegaPonyXLAnime_v20 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/omegaPonyXLAnime_v20.safetensors")

Illustrious_XL_v01 = False  #@param {type:"boolean"}
Illustrious_XL_v01 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors")

Illustrious_XL_v10 = False  #@param {type:"boolean"}
Illustrious_XL_v10 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v1.0.safetensors")

Illustrious_XL_v11 = False  #@param {type:"boolean"}
Illustrious_XL_v11 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v1.1.safetensors")

Illustrious_XL_v20_Stable = True  #@param {type:"boolean"}
Illustrious_XL_v20_Stable and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v2.0-stable.safetensors")

noobaiXL_EpsV01 = False  #@param {type:"boolean"}
noobaiXL_EpsV01 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_earlyAccessVersion.safetensors")

noobaiXL_EpsV05 = False  #@param {type:"boolean"}
noobaiXL_EpsV05 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred05Version.safetensors")

noobaiXL_EpsV075 = False  #@param {type:"boolean"}
noobaiXL_EpsV075 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred075.safetensors")

noobaiXL_EpsV077 = False  #@param {type:"boolean"}
noobaiXL_EpsV077 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred077.safetensors")

noobaiXL_EpsV10 = False  #@param {type:"boolean"}
noobaiXL_EpsV10 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred10Version.safetensors")

noobaiXL_EpsV11 = False  #@param {type:"boolean"}
noobaiXL_EpsV11 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred11Version.safetensors")

noobaiXL_VPre01 = False  #@param {type:"boolean"}
noobaiXL_VPre01 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPredTestVersion.safetensors")

noobaiXL_VPre05 = False  #@param {type:"boolean"}
noobaiXL_VPre05 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred05Version.safetensors")

noobaiXL_VPre06 = False  #@param {type:"boolean"}
noobaiXL_VPre06 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred06Version.safetensors")

noobaiXL_VPre065S = False  #@param {type:"boolean"}
noobaiXL_VPre065S and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred065SVersion.safetensors")

noobaiXL_VPre075S = False  #@param {type:"boolean"}
noobaiXL_VPre075S and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred075SVersion.safetensors")

noobaiXL_VPre09R = False  #@param {type:"boolean"}
noobaiXL_VPre09R and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred09RVersion.safetensors")

noobaiXL_VPre10 = True  #@param {type:"boolean"}
noobaiXL_VPre10 and fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred10Version.safetensors")


##############################
CLEAR_LOG and fooocus.utils.clear_up()
logger.info("模型下载完成")

In [ ]:
#@title 👇 自定义模型下载
try:
    _ = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

#@markdown ### 选择模型种类：
model_type = "LoRA" # @param ["Stable Diffusion", "LoRA", "VAE", "Embeding"]
#@markdown ### 填写模型的下载链接：
model_url = "https://huggingface.co/licyk/sd-lora/resolve/main/sdxl/style/CoolFlatColor.safetensors"  #@param {type:"string"}
#@markdown ### 填写模型的名称（包括后缀名）：
model_name = "CoolFlatColor.safetensors"  #@param {type:"string"}

if model_type == "Stable Diffusion":
    fooocus.get_sd_model(url=model_url, filename=model_name or None)
elif model_type == "LoRA":
    fooocus.get_lora_model(url=model_url, filename=model_name or None)
elif model_type == "VAE":
    fooocus.get_vae_model(url=model_url, filename=model_name or None)
elif model_type == "Embeding":
    fooocus.get_embedding_model(url=model_url, filename=model_name or None)

In [ ]:
#@title 👇 启动 Fooocus
try:
    _ = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

logger.info("启动 Fooocus")
os.chdir(FOOOCUS_PATH)
fooocus.mount_drive()
fooocus.tun.start_tunnel(**TUNNEL_PARAMS)
logger.info("Fooocus 启动参数: %s", FOOOCUS_LAUNCH_ARGS)
if USE_GRADIO:
    !python "{FOOOCUS_PATH}/launch.py" {FOOOCUS_LAUNCH_ARGS} --share
else:
    !python "{FOOOCUS_PATH}/launch.py" {FOOOCUS_LAUNCH_ARGS}
CLEAR_LOG and fooocus.utils.clear_up()
logger.info("已关闭 Fooocus")